In [1]:
import pandas as pd
from datetime import datetime, timedelta
import os
from tqdm import tqdm
import pickle
tqdm.pandas()
import warnings
import catboost as cb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
# from submission_xgboost.utils import create_features_and_target
# from submission_xgboost.utils import create_features_for_prediction
warnings.filterwarnings("ignore")

C:\Users\isaac\miniconda3\envs\orekit_venv_v12\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Load Initial position and dict of Omni

In [2]:
initial_states = pd.read_csv('C:/Users/isaac/Documents/Challenge_MIT_2/mit_challenge_2/data/phase_1/00000_to_02284-initial_states.csv')
initial_states = pd.concat([initial_states,pd.read_csv('C:/Users/isaac/Documents/Challenge_MIT_2/mit_challenge_2/data/phase_1/02285_to_02357-initial_states.csv')])
initial_states = pd.concat([initial_states,pd.read_csv('C:/Users/isaac/Documents/Challenge_MIT_2/mit_challenge_2/data/phase_1/02358_to_04264-initial_states.csv')])
initial_states = pd.concat([initial_states,pd.read_csv('C:/Users/isaac/Documents/Challenge_MIT_2/mit_challenge_2/data/phase_1/04265_to_05570-initial_states.csv')])
initial_states = pd.concat([initial_states,pd.read_csv('C:/Users/isaac/Documents/Challenge_MIT_2/mit_challenge_2/data/phase_1/05571_to_05614-initial_states.csv')])
initial_states = pd.concat([initial_states,pd.read_csv('C:/Users/isaac/Documents/Challenge_MIT_2/mit_challenge_2/data/phase_1/05615_to_06671-initial_states.csv')])
initial_states = pd.concat([initial_states,pd.read_csv('C:/Users/isaac/Documents/Challenge_MIT_2/mit_challenge_2/data/phase_1/06672_to_08118-initial_states.csv')])
initial_states['Timestamp'] = pd.to_datetime(initial_states['Timestamp'])

initial_states = initial_states.set_index('File ID')
print(initial_states.shape)
initial_states =  initial_states.mask((initial_states==9.99e+29)|(initial_states==9.99e+32))
initial_states = initial_states.loc[initial_states.count(1)==10]
print(initial_states.shape)

(8119, 10)
(8095, 10)


In [4]:
with open('data/data_isaac/omni.pickle', 'rb') as f:
    omni = pickle.load(f)
with open('data/data_isaac/sat.pickle', 'rb') as f:
    sat = pickle.load(f)

In [156]:
for i in sat.keys():
    sat[i]['Orbit Mean Density (kg/m^3)'] = sat[i]['Orbit Mean Density (kg/m^3)'].mask(sat[i]['Orbit Mean Density (kg/m^3)']>1_000_000)
    sat[i]['Orbit Mean Density (kg/m^3)'] = sat[i]['Orbit Mean Density (kg/m^3)'].interpolate()

In [ ]:
# all_files_omni = os.listdir("phase_1/omni2/")
# all_files_dens = os.listdir("phase_1/sat_density/")
# omni = {}
# for i in tqdm(all_files_omni):
#     omni[int(i.split('-')[1])] = pd.read_csv('phase_1/omni2/'+i)
# sat = {}
# for i in tqdm(all_files_dens):
#     sat[int(i.replace('gr-of','grof').split('-')[1])] = pd.read_csv('phase_1/sat_density/'+i)
# with open('omni.pickle', 'wb') as handle:
#     pickle.dump(omni, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('sat.pickle', 'wb') as handle:
#     pickle.dump(sat, handle, protocol=pickle.HIGHEST_PROTOCOL)


###  Model by MSIS

In [24]:
from org.orekit.frames import FramesFactory
from org.orekit.utils import Constants
from org.orekit.utils import IERSConventions
from org.orekit.bodies import OneAxisEllipsoid
from org.orekit.orbits import KeplerianOrbit, OrbitType
import math

In [3]:
from submission_presistence.atm_me import PersistenceMSIS, MSISPersistenceAtmosphere, PersistenceModel

In [141]:
model = PersistenceModel(plot_trajectory=False)
def get_new_position_of_satellite(initial_state,omni_data_refined,model):
    states = model(None,initial_state)
    positions = pd.DataFrame([extract_kepllerian_parameters(i) for i in states])
    return positions
    

In [163]:
res = {}
for  i in  tqdm(initial_states.index):
    res[i] = get_new_position_of_satellite(initial_states.loc[i].to_dict(),None,model) 

100%|████████████████████████████████████████████████████████████████████████████| 8095/8095 [2:39:27<00:00,  1.18s/it]


In [190]:
with open('data/data_isaac/sattelite_estimated_position.pickle', 'wb') as handle:
     pickle.dump(res, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [189]:
for  i in  tqdm(initial_states.index):
    res[i]['Timestamp'] = res[i]['Timestamp'].apply(lambda win:win.toString())

100%|█████████████████████████████████████████████████████████████████████████████| 8095/8095 [00:50<00:00, 161.26it/s]


In [195]:
df_data = pd.merge(pd.concat(res),pd.concat(sat),left_index=True,right_index=True)

In [202]:
df_data

A         E          I        RAAN          PA  \
0    0    6.826387e+06  0.003882  87.275306  144.135111 -102.685611   
     1    6.819094e+06  0.004154  87.273526  144.134319 -117.762208   
     2    6.808346e+06  0.005755  87.271238  144.130605  250.187879   
     3    6.810499e+06  0.004984  87.271667  144.125700  265.448583   
     4    6.822372e+06  0.003626  87.273897  144.123157  251.662548   
...                ...       ...        ...         ...         ...   
8118 139  6.761777e+06  0.008672  87.482044   92.194018   99.002279   
     140  6.773385e+06  0.006932  87.484408   92.190896   97.516854   
     141  6.778605e+06  0.007311  87.485587   92.190443   90.683569   
     142  6.768749e+06  0.007072  87.483825   92.189368  101.406877   
     143  6.759225e+06  0.005286  87.482044   92.185215   98.205895   

                  TA               Timestamp_x  latitudeDeg  longitudeDeg  \
0    0    102.383270  2000-08-02T04:50:33.000Z    -0.305782    120.394686   
     1    155.789728  2000-08-02T05:00:33.000Z    38.148777    120.031970   
     2   -173.947157  2000-08-02T05:10:33.000Z    76.059805    126.392297   
     3   -150.980085  2000-08-02T05:20:33.000Z    65.526243    -73.093176   
     4    -98.821246  2000-08-02T05:30:33.000Z    27.273377    -71.028112   
...              ...                       ...          ...           ...   
8118 139 -169.444965  2019-12-31T23:10:00.000Z   -70.392476     -2.490822   
     140 -129.491287  2019-12-31T23:20:00.000Z   -32.103872      0.715762   
     141  -83.889319  2019-12-31T23:30:00.000Z     6.824794      0.158610   
     142  -55.482207  2019-12-31T23:40:00.000Z    46.039312      0.046641   
     143  -12.886631  2019-12-31T23:50:00.000Z    84.663272     24.043983   

               altitude          Timestamp_y  Orbit Mean Density (kg/m^3)  
0    0    453835.680651  2000-08-02 04:50:40                 1.803589e-12  
     1    474894.137697  2000-08-02 05:00:40                 1.812230e-12  
     2    489302.436401  2000-08-02 05:10:40                 1.823796e-12  
     3    479691.892475  2000-08-02 05:20:40                 1.827751e-12  
     4    452402.938332  2000-08-02 05:30:40                 1.787903e-12  
...                 ...                  ...                          ...  
8118 139  460228.738068  2019-12-31 23:10:00                 1.359338e-13  
     140  430921.340346  2019-12-31 23:20:00                 1.465138e-13  
     141  395135.124586  2019-12-31 23:30:00                 1.456536e-13  
     142  374295.177610  2019-12-31 23:40:00                 1.409124e-13  
     143  367446.521995  2019-12-31 23:50:00                 1.397193e-13  

[3480343 rows x 12 columns]

In [135]:
# id = 102
# omni_data = omni[id].loc[:,['Timestamp', 'f10.7_index', 'ap_index_nT']]
# omni_data['Timestamp'] = pd.to_datetime(omni_data['Timestamp'])
# omni_data = omni_data.ffill()
# states = model(omni_data,initial_states.loc[id].to_dict())


In [ ]:
r_Earth = Constants.IERS2010_EARTH_EQUATORIAL_RADIUS  # m
# International Terrestrial Reference Frame, earth fixed
itrf = FramesFactory.getITRF(IERSConventions.IERS_2010, True)
earth = OneAxisEllipsoid(
    r_Earth,
    Constants.IERS2010_EARTH_FLATTENING,
    itrf
)

In [134]:
def position_to_geo(pvEME2000, date):
    """
    Converts a position vector (in EME2000 frame) to geodetic coordinates (lat, lon, alt).

    Parameters:
    positionICRF: Vector3D, position vector in ICRF frame.
    date: AbsoluteDate, the date of the position.

    Returns:
    tuple: (latitude, longitude, altitude) in degrees and meters.
    """
    # Transform from EME200 to ICRF
    pvICRF = inertial_frame.getTransformTo(earth_frame,date).transformPVCoordinates(pvEME2000)

    # Transform position from ICRF to ECEF (ITRF)
    transform = earth.getBodyFrame().getTransformTo(itrf, date)
    pvECEF = transform.transformPVCoordinates(pvICRF)
    positionECEF = pvECEF.getPosition()

    # Convert the ECEF position to geodetic coordinates
    geodeticPoint = earth.transform(positionECEF, itrf, date)

    # Extract latitude, longitude, and altitude
    latitude = geodeticPoint.getLatitude()  # radians
    longitude = geodeticPoint.getLongitude()  # radians
    altitude = geodeticPoint.getAltitude()  # meters

    # Convert radians to degrees for latitude and longitude
    latitudeDeg = math.degrees(latitude)
    longitudeDeg = math.degrees(longitude)

    return latitudeDeg, longitudeDeg, altitude

In [133]:
def extract_kepllerian_parameters(sat_state):
    r = sat_state.orbit
    converted_orbit = KeplerianOrbit.cast_(OrbitType.KEPLERIAN.convertType(r))
    latitudeDeg, longitudeDeg, altitude = position_to_geo(sat_state.getPVCoordinates(), sat_state.date)
    res ={
        'A':converted_orbit.getA(),
        'E':converted_orbit.getE(),
    'I':converted_orbit.getI()* 180.0 / math.pi,
    'RAAN':converted_orbit.getRightAscensionOfAscendingNode()* 180.0 / math.pi ,
    'PA':converted_orbit.getPerigeeArgument()* 180.0 / math.pi ,
    'TA':converted_orbit.getTrueAnomaly()* 180.0 / math.pi,
    'Timestamp':r.date,
    'latitudeDeg' : latitudeDeg, 'longitudeDeg':longitudeDeg, 'altitude': altitude}

    
    return res

In [78]:
inertial_frame = FramesFactory.getEME2000()
earth_frame = FramesFactory.getITRF(IERSConventions.IERS_2010, False)
pv = a.getPVCoordinates()
pv_itrf = inertial_frame.getTransformTo(earth_frame, a.date).transformPVCoordinates(pv)

In [87]:
inertial_frame

<FactoryManagedFrame: EME2000>

In [90]:
earth.getBodyFrame()

<Frame: CIO/2010-based ITRF simple EOP>

In [86]:
earth_frame

<FactoryManagedFrame: CIO/2010-based ITRF accurate EOP>

In [88]:
itrf

<FactoryManagedFrame: CIO/2010-based ITRF simple EOP>

In [65]:
from org.orekit.time import AbsoluteDate
from org.orekit.utils import PVCoordinates, Constants
from org.orekit.frames import Frame
from org.orekit.models.earth.atmosphere import PythonAtmosphere
from org.hipparchus.geometry.euclidean.threed import Vector3D
from org.orekit.frames import FramesFactory, Frame
from org.orekit.bodies import OneAxisEllipsoid
from org.orekit.utils import IERSConventions

In [52]:
a= states[0]

In [71]:
initial_states.loc[id]

Timestamp                    2001-02-14 01:27:22
Semi-major Axis (km)                 6815.861229
Eccentricity                            0.003726
Inclination (deg)                      87.263806
RAAN (deg)                             70.763889
Argument of Perigee (deg)             212.857694
True Anomaly (deg)                    147.039955
Latitude (deg)                         30.460832
Longitude (deg)                        82.810318
Altitude (km)                         428.642907
Name: 102, dtype: object

In [131]:
a

<SpacecraftState: SpacecraftState{orbit=Keplerian parameters: {a: 6815861.2294501355; e: 0.0037259000819176; i: 87.26380555555555; pa: 212.85769444444443; raan: 70.76388888888889; v: 147.03995549397703;}, attitude=org.orekit.attitudes.Attitude@56a9a7b5, mass=260.0, additional={}, additionalDot={}}>

In [127]:
inertial_frame.getTransformTo()

<FactoryManagedFrame: EME2000>

In [130]:
a

<SpacecraftState: SpacecraftState{orbit=Keplerian parameters: {a: 6815861.2294501355; e: 0.0037259000819176; i: 87.26380555555555; pa: 212.85769444444443; raan: 70.76388888888889; v: 147.03995549397703;}, attitude=org.orekit.attitudes.Attitude@56a9a7b5, mass=260.0, additional={}, additionalDot={}}>

(-0.10197853150343263, -95.211616939038, 459004.4637111966)

In [97]:
pd.DataFrame([extract_kepllerian_parameters(states[i]) for i in range(len(states))])

,A,E,I,RAAN,PA,TA,Timestamp
0,6.815861e+06,0.003726,87.263806,70.763889,212.857694,147.039955,2001-02-14T01:27:22.000Z
1,6.808568e+06,0.004634,87.262224,70.763155,204.597638,193.598648,2001-02-14T01:37:22.000Z
2,6.797806e+06,0.005585,87.260000,70.759493,221.160530,215.355933,2001-02-14T01:47:22.000Z
3,6.800102e+06,0.004142,87.260485,70.754469,230.400336,244.581960,2001-02-14T01:57:22.000Z
4,6.812149e+06,0.003815,87.262934,70.751764,205.981087,307.680477,2001-02-14T02:07:22.000Z
...,...,...,...,...,...,...,...
427,6.797395e+06,0.004476,87.260483,69.638557,173.917992,16667.534788,2001-02-17T00:37:22.000Z
428,6.807961e+06,0.004781,87.262566,69.634839,195.551438,16684.426161,2001-02-17T00:47:22.000Z
429,6.815841e+06,0.003696,87.264023,69.634091,200.539328,16717.799809,2001-02-17T00:57:22.000Z
430,6.809070e+06,0.004622,87.262506,69.633426,193.995296,16762.638094,2001-02-17T01:07:22.000Z


### Test MSIS

In [ ]:
model = PersistenceModel(plot_trajectory=False)
for id in df_train.index:
    if os.path.exists('data_isaac/sat_density_pred_no_drag/density_pred'+ str(id) + '.csv'):
        continue
    else:
        bad_id.append(id)
        continue
    omni_data = omni[id].loc[:,['Timestamp', 'f10.7_index', 'ap_index_nT']]
    omni_data['Timestamp'] = pd.to_datetime(omni_data['Timestamp'])
    omni_data = omni_data.ffill()
    try:
        states, densities = model(omni_data,df_train.loc[id].to_dict())
        predictions = model._convert_to_df(states, densities)
        predictions.to_csv('data_isaac/sat_density_pred_no_drag/density_pred'+ str(id) + '.csv')
    except:
        print(id)